# Fine-tuned BERT News Summarizer
### Jingfeng Xia

In [ ]:
!pip install transformers torch prettytable bert-extractive-summarizer datasets rouge_score rouge

In [ ]:
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datasets import load_dataset
import re
import transformers
from transformers import BertTokenizer, BertModel, EncoderDecoderModel
import torch
from tqdm import tqdm_notebook as tqdm
import datasets
from transformers import TrainingArguments
from transformers import Trainer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional
import datasets
from difflib import SequenceMatcher

In [ ]:
import pandas as pd
import numpy as np
import transformers
from transformers import BertTokenizer, BertModel, EncoderDecoderModel
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

# Data Preprocessing

In [ ]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def save_json(df,filename):
    df.to_json(filename)

def read_json(filename):
    return pd.read_json(filename)
seed_val=123

In [ ]:
raw=pd.read_csv('NYT_Dataset.csv')

In [ ]:
# raw=pd.read_csv('NYT_Dataset.csv')
# # raw.head()
# mid=raw["abstract"].map(str)+' '+raw["keywords"].map(str)
# raw['combination']=mid.apply(lambda text: normalize_text(text))
# raw['clean_title']=raw['title'].map(str).apply(lambda text: normalize_text(text))
# new=pd.DataFrame()
# new['input']=raw['combination'].values
# new['output']=raw['clean_title'].values

In [ ]:
# save_json(new,'full_data.json')
full_data=read_json('full_data.json')

In [ ]:
# import matplotlib.pyplot as plt
# hist_1=full_data['input'].str.len().hist(bins=10)
# plt.suptitle("Hist for Text")
# plt.xlabel("sentence length")
# plt.ylabel("num of instances")

In [ ]:
# hist_1=full_data['output'].str.len().hist(bins=10)
# plt.suptitle("Hist for Summary")
# plt.xlabel("sentence length")
# plt.ylabel("num of instances")

In [ ]:
# from prettytable import PrettyTable
# input_num=len(full_data['input'])
# output_num=len(full_data['output'])
# input_max=full_data['input'].str.len().max()
# output_max=full_data['output'].str.len().max()
# input_min=full_data['input'].str.len().min()
# output_min=full_data['output'].str.len().min()
# input_95=int(np.percentile(full_data['input'].str.len(),95))
# output_95=int(np.percentile(full_data['output'].str.len(),95))
# input_5=int(np.percentile(full_data['input'].str.len(),5))
# output_5=int(np.percentile(full_data['output'].str.len(),5))
# # print(' input_max_len:',input_max,'\n','output_max_len:',output_max,'\n',\
# #       'input_min_len:',input_min,'\n','output_min_len:',output_min,'\n',\
# #       '95 percentile input length:',input_95,'\n','95 percentile output length:',output_95,'\n',\
# #       '5 percentile input length:',input_5,'\n','5 percentile output length:',output_5,'\n')

# x = PrettyTable()
# x.field_names = ["measure","text", "summary"]
# x.add_rows(
#     [
#         ["num_inst", input_num, output_num],
#         ["max_len", input_max, output_max],
#         ["min_len", input_min,output_min],
#         ["95_percnt", input_95, output_95],
#         ["5_percnt", input_5, output_5]
#     ]
# )
# print(x)

In [ ]:
# sample 1000 instances
part_data=full_data.sample(100,random_state=seed_val)

In [ ]:
# from sklearn.model_selection import train_test_split
# def tr_te_split(dataset_name):
#     train_x, test_x, train_y, test_y = train_test_split(dataset_name.drop('output', axis = 1),dataset_name['output'],test_size = 0.3,random_state = seed_val)
#     return train_x, test_x, train_y, test_y

# Pretrained BERT Summarizer

In [ ]:
example = '''
       Scientists say they have discovered a new species of orangutans on Indonesia’s island of Sumatra.
The population differs in several ways from the two existing orangutan species found in Sumatra and the neighboring island of Borneo.
The orangutans were found inside North Sumatra’s Batang Toru forest, the science publication Current Biology reported.
Researchers named the new species the Tapanuli orangutan. They say the animals are considered a new species because of genetic, skeletal and tooth differences.
Michael Kruetzen is a geneticist with the University of Zurich who has studied the orangutans for several years. He said he was excited to be part of the unusual discovery of a new great ape in the present day. He noted that most great apes are currently considered endangered or severely endangered.
Gorillas, chimpanzees and bonobos also belong to the great ape species.
Orangutan – which means person of the forest in the Indonesian and Malay languages - is the world’s biggest tree-living mammal. The orange-haired animals can move easily among the trees because their arms are longer than their legs. They live more lonely lives than other great apes, spending a lot of time sleeping and eating fruit in the forest.
The new study said fewer than 800 of the newly-described orangutans exist. Their low numbers make the group the most endangered of all the great ape species.
They live within an area covering about 1,000 square kilometers. The population is considered highly vulnerable. That is because the environment which they depend on is greatly threatened by development.
Researchers say if steps are not taken quickly to reduce the current and future threats, the new species could become extinct “within our lifetime.”
Research into the new species began in 2013, when an orangutan protection group in Sumatra found an injured orangutan in an area far away from the other species. The adult male orangutan had been beaten by local villagers and died of his injuries. The complete skull was examined by researchers.
Among the physical differences of the new species are a notably smaller head and frizzier hair. The Tapanuli orangutans also have a different diet and are found only in higher forest areas.
There is no unified international system for recognizing new species. But to be considered, discovery claims at least require publication in a major scientific publication.
Russell Mittermeier is head of the primate specialist group at the International Union for the Conservation of Nature. He called the finding a “remarkable discovery.” He said it puts responsibility on the Indonesian government to help the species survive.
Matthew Nowak is one of the writers of the study. He told the Associated Press that there are three groups of the Tapanuli orangutans that are separated by non-protected land.He said forest land needs to connect the separated groups.
In addition, the writers of the study are recommending that plans for a hydropower center in the area be stopped by the government.
It also recommended that remaining forest in the Sumatran area where the orangutans live be protected.
I’m Bryan Lynn.

        '''

In [ ]:
from summarizer import Summarizer
model = Summarizer()
result = model(example,min_length=2,max_length=75)
summ = "".join(result)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(summ)

Researchers named the new species the Tapanuli orangutan. The new study said fewer than 800 of the newly-described orangutans exist. There is no unified international system for recognizing new species.


#Fine tune BERT Summarizer

In [ ]:
#The tokenizer to be used to create embeddings for both Articles and Summaries
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
batch_size = 16 # Change batch size to 4 for faster but less accurate training
encoder_max_length = 512
decoder_max_length = 128

def convert_data_to_model_inputs(part_data):
    
    #Encodes the article
    part_data['itk']=part_data['input'].apply(lambda text: tokenizer(text, padding="max_length", truncation=True, max_length = encoder_max_length))
    #Encodes the summary
    part_data['otk']=part_data['output'].apply(lambda text: tokenizer(text, padding="max_length", truncation=True, max_length = encoder_max_length))
    part_data['input_ids']=part_data['itk'].apply(lambda x: x.input_ids)
    part_data['attention_mask']=part_data['itk'].apply(lambda x: x.attention_mask)
    part_data['decoder_input_ids']=part_data['otk'].apply(lambda x: x.input_ids)
    part_data['decoder_attention_mask']=part_data['otk'].apply(lambda x: x.attention_mask)
    part_data['labels']=part_data['otk'].apply(lambda x: x.input_ids.copy())

    return part_data

In [ ]:
data=convert_data_to_model_inputs(part_data)[["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"]]

In [ ]:
tr_data = data.iloc[:int(0.7*len(data)), :]
vl_data = data.iloc[int(0.7*len(data)):, :]

In [ ]:
tr_data["input_ids"],tr_data["attention_mask"],tr_data["decoder_input_ids"],tr_data["decoder_attention_mask"],tr_data["labels"]
vl_data["input_ids"],vl_data["attention_mask"],vl_data["decoder_input_ids"],vl_data["decoder_attention_mask"],vl_data["labels"]

In [ ]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, decoder_input_ids, decoder_attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.decoder_input_ids = decoder_input_ids
        self.decoder_attention_mask = decoder_attention_mask
        self.labels = labels

    def __getitem__(self, idx):
        item["input_ids"] = torch.tensor([self.input_ids[idx]])
        item["attention_mask"] = torch.tensor([self.attention_mask[idx]])
        item["decoder_input_ids"] = torch.tensor([self.decoder_input_ids[idx]])
        item["decoder_attention_mask"] = torch.tensor([self.decoder_attention_mask[idx]])
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_data = NewsGroupsDataset(tr_data["input_ids"],tr_data["attention_mask"],tr_data["decoder_input_ids"],tr_data["decoder_attention_mask"],tr_data["labels"])
val_dataset = NewsGroupsDataset(vl_data["input_ids"],vl_data["attention_mask"],vl_data["decoder_input_ids"],vl_data["decoder_attention_mask"],vl_data["labels"])

In [ ]:
#Creates the base encoder decoder model
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased","bert-base-uncased")

# set model configuration
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
model.config.vocab_size = model.config.decoder.vocab_size
model.config.max_length = 128
model.config.min_length = 64
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [ ]:
import datasets
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output1 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
    rouge_output2 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge1_precision": round(rouge_output1.precision, 4),
        "rouge1_recall": round(rouge_output1.recall, 4),
        "rouge1_fmeasure": round(rouge_output1.fmeasure, 4),
        "rouge2_precision": round(rouge_output2.precision, 4),
        "rouge2_recall": round(rouge_output2.recall, 4),
        "rouge2_fmeasure": round(rouge_output2.fmeasure, 4),
    }

In [ ]:
#Frees up unused memory 
import gc
torch.cuda.empty_cache()
gc.collect()

2996

In [ ]:
torch.tensor(targets_df['targets'].values)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    evaluation_strategy='steps',
    do_train=True,
    do_eval=True,
    logging_steps=1000,
    save_steps=500, 
    eval_steps=8000,
    warmup_steps=2000,  
    overwrite_output_dir=True,
    save_total_limit=3,
    # fp16=True, #Comment out this line if training a non-CUDA device
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 70
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 15


KeyError: ignored

# Evaluation

In [ ]:
## metrics: Bertscore, rouge, bleurt

text=part_data['input'].iloc[2]
gold=part_data['output'].iloc[2]
from summarizer import Summarizer
model = Summarizer()
result = model(text,min_length=2,max_length=75)
summ = "".join(result)
# print(summ)
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(text, summ)
print(scores)